<a href="https://colab.research.google.com/github/mmistroni/jupyter/blob/master/NewsSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas OpenBlender scikit-learn
!pip install vaderSentiment

<p> Not really applicable. We need to interact back and forth with OpenBlender. Trying Alternate API </p>

<h3> NewsAPI good but we have a maximum limit</h3>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

def get_iexapi_keys():
  with open('gdrive/My Drive/passwords/iexapi.keys') as f:
    return f.readlines()[0]

def get_nlp_service_keys():
  with open('gdrive/My Drive/passwords/nlp.keys') as f:
    return f.readlines()[0]

def get_newsapi_keys():
  with open('gdrive/My Drive/passwords/newsapi.keys') as f:
    return f.readlines()[0]

In [ ]:
import requests
def retrieve_all_news(symbol):
  print('Retrieving all news for@{}'.format(symbol))
  token = get_newsapi_keys()
  all_news = 'https://newsapi.org/v2/everything?q={ticker}&apiKey={token}'.format(ticker=symbol, token=get_newsapi_keys())
  data = requests.get(all_news).json()
  res = data['articles']
  return map(lambda data: data['content'], res)

<h3> Fetching news from finviz (good but only applies to daily news )</h3>

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, date
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pandas as pd
from pandas.tseries.offsets import BDay
# This is good one, Free

def get_news_from_finviz2(tickers):
  news_tables = {}
  for ticker in tickers:
    url = 'https://finviz.com/quote.ashx?t={}'.format(ticker)
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    response = urlopen(req)    
    html = BeautifulSoup(response)
    print('Now parsing...')
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table
    
  parsed_news = []
  print('Found:{}'.format(len(news_tables.items())))
  for file_name, news_table in news_tables.items():
      for x in news_table.findAll('tr'):
          text = x.a.get_text() 
          ticker = file_name.split('_')[0]
          parsed_news.append([ticker, date, time, text])
  return parsed_news



def get_news_from_finviz(tickers):
  news_tables = {}
  for ticker in tickers:
    url = 'https://finviz.com/quote.ashx?t={}'.format(ticker)
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    response = urlopen(req)    
    html = BeautifulSoup(response)
    print('Now parsing...')
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table
    
  parsed_news = []
  print('Found:{}'.format(len(news_tables.items())))
  for file_name, news_table in news_tables.items():
      for x in news_table.findAll('tr'):
      
          text = x.a.get_text() 
          date_scrape = x.td.text.split()
          link = x.a.get('href')
          if len(date_scrape) == 1:
              time = date_scrape[0]
          else:
              date = date_scrape[0]
              time = date_scrape[1]
          ticker = file_name.split('_')[0]
          
          parsed_news.append([ticker, date, time, text, link])
  return parsed_news

def get_sentiment_from_vader(sentence):
    analyser = SentimentIntensityAnalyzer()
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))
    return score['compound']


def find_news_scores_for_ticker(tickers):
  try:
    parsed_news = get_news_from_finviz(tickers)
    #mapped_dt = list(map(lambda tpl:tpl[1], parsed_news))
    #return mapped_dt
  except Exception as e:
    print('cant find naything for :{}'.format(tickers))
    return []

  columns = ['ticker', 'date', 'time', 'headline', 'link']
  parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)
  parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date
  yday = date.today() - BDay(20)
  filtered = parsed_and_scored_news[parsed_and_scored_news['date'] > yday]
  #print(parsed_and_scored_news[['date', 'ticker', 'headline']].head())
  parsed_and_scored_news = filtered.groupby(['ticker'], as_index = False).agg({'headline': ''.join}, Inplace=True)
  scores = parsed_and_scored_news['headline'].apply(get_sentiment_from_vader).tolist()
  scores_df = pd.DataFrame(scores)
  parsed_and_scored_news = parsed_and_scored_news.join(scores_df, rsuffix='_right')
  return parsed_and_scored_news


r = find_news_scores_for_ticker(['PTON'])
print(type(r))
r

#get_sentiment_from_vader("Here's what booming dollar store sales say about America right now Yahoo Finance")


In [ ]:
# getting all sectors all companies
from google.colab import auth
auth.authenticate_user()
!mkdir -p data
!gsutil cp gs://datascience-bucket-mm/all_sectors.csv data
sct = pd.read_csv('data/all_sectors.csv')
sct.rename({'Unnamed: 0' : 'ticker', 'Sector':'sector'}, axis=1, inplace=True)
print(sct.shape)



In [ ]:
sct['sector'].unique()

In [ ]:
ccycles = sct[sct['sector'] == 'Utilities']
ccycles.shape

In [ ]:
all_data = dict((tick,find_news_scores_for_ticker([tick]) ) for tick in ccycles.ticker.values.tolist())

In [ ]:
all_df = [df for df in all_data.values() if isinstance(df,pd.DataFrame)]
r = pd.concat(all_df, axis=0)
r.shape
r.head(3)

In [ ]:
tst = all_data['D']
tst


In [ ]:
d = tst.to_dict()
dict((k, d[k][0]) for k in d.keys())

In [ ]:
import _pickle as pickle
with open("tick_to_news_august.pkl", "wb") as fp:
    pickle.dump(all_data, fp)

In [ ]:
r.to_csv('utilities_news.csv')
!gsutil cp 'utilities_news.csv' gs://datascience-bucket-mm/

In [ ]:
!gsutil cp  gs://datascience-bucket-mm/tick_to_news.pkl .

In [ ]:
import _pickle as pickle
with open("tick_to_news.pkl",'rb') as ts:
  new_dict = pickle.load(ts)
  

In [ ]:
len(new_dict.items())

In [ ]:
all_x = map(lambda tpl: (tpl[0],tpl[-1] if tpl[1] else None), new_dict.items())

In [ ]:
first = list(all_x)[0:10]
first

In [ ]:
list(new_dict.keys())[0:2]
for k in list(new_dict.keys())[0:10]:
  print(new_dict[k])

In [ ]:
list_of_dict = [(k, ','.join(list(v)) if list(v) else None) for k, v in all_data.items()]
df = pd.DataFrame(list_of_dict, columns=['ticker', 'oldest_news'])

In [ ]:
df.to_csv("all_ticker_news.csv")
!gsutil cp all_ticker_news.csv gs://datascience-bucket-mm/


In [ ]:
!gsutil cp gs://datascience-bucket-mm/all_ticker_news.csv .

In [ ]:
import pandas as pd
df = pd.read_csv('all_ticker_news.csv')
df.shape



In [ ]:
df['oldest_news']

<h3>Historical news </h3

<h4> Trying first iexcloud and see how much we use </h3>

<h3> Using BigQuery to estract dat aint oPandas </h3>

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
from google.cloud import bigquery
import numpy as np
import tensorflow as tf
import pandas as pd
pd.options.mode.chained_assignment = None
import seaborn as sns
from matplotlib.pylab import rcParams
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline

client = bigquery.Client('datascience-projects')

In [ ]:
sql = """
SELECT *
FROM
    `datascience-projects.gcp_edgar.form_13hf_daily`

"""
df = client.query(sql).to_dataframe()
df.head(10)

In [ ]:
df[['TICKER', 'CUSIP','COUNT']].groupby(['TICKER','CUSIP',  'COUNT']).sum().sort_values(by='COUNT', ascending=False).head(30)

In [ ]:
filtered = df[df['TICKER'] == 'MU'].sort_values(by='COB')
import matplotlib.pyplot as plt
import pandas as pd
fig = go.Figure()
fig.add_trace(go.Scatter(x=filtered.COB, y=filtered.COUNT,
                    mode='lines',
                    name='close'))
#fig.update_layout(showlegend=True)
#fig.show()
filtered



<h3> Fetching analyst recommendations </h3>

In [ ]:
import requests
recommendations = []
for ticker in ['AAPL', 'AMZN']:
    lhs_url = 'https://query2.finance.yahoo.com/v10/finance/quoteSummary/'
    rhs_url = '?formatted=true&crumb=swg7qs5y9UP&lang=en-US&region=US&' \
              'modules=upgradeDowngradeHistory,recommendationTrend,' \
              'financialData,earningsHistory,earningsTrend,industryTrend&' \
              'corsDomain=finance.yahoo.com'
              
    url =  lhs_url + ticker + rhs_url
    r = requests.get(url)
    if not r.ok:
        recommendation = 6
    try:
        result = r.json()['quoteSummary']['result'][0]
        recommendation =result['financialData']['recommendationMean']['fmt']
    except:
        recommendation = 6
    
    recommendations.append(recommendation)

In [ ]:
recommendations